In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [80]:
import pandas as pd
df_dev = pd.read_csv('drive/MyDrive/Colab Notebooks/development.tsv',sep = '\t')
df_eval = pd.read_csv('drive/MyDrive/Colab Notebooks/evaluation.tsv',sep = '\t')

In [81]:
df_dev.head()

,beer/ABV,beer/name,beer/style,review/appearance,review/aroma,review/overall,review/palate,review/taste,review/text,user/ageInSeconds,user/birthdayRaw,user/birthdayUnix,user/gender,user/profileName
0,5.70,Barney Flats Oatmeal Stout,Oatmeal Stout,4.5,5.0,4.5,4.0,4.5,Pours a dark almost cola color with reddish br...,1.066918e+09,"Feb 19, 1981",351417600.0,Male,goblue3509
1,9.99,Brown Shugga',American Strong Ale,4.0,4.0,4.0,4.5,5.0,I went to a place called World Market in Oxnar...,NaN,NaN,NaN,NaN,Lounge14ABV
2,5.35,Casta Bruna (Pale Ale),English Pale Ale,3.0,3.0,4.0,3.0,3.5,I clicked on Casta tonight just to add some no...,NaN,NaN,NaN,NaN,DogFood11
3,8.00,Old Engine Oil Special Reserve (Aged In Malt W...,Old Ale,4.0,4.5,4.0,5.0,4.0,Wonderful little beer. Pours a deep brown with...,NaN,NaN,NaN,NaN,BasementDweller
4,7.00,Dark Humor,American Porter,3.5,4.0,4.0,4.0,4.5,Excited to see this on tap at Fathead's origin...,NaN,NaN,NaN,NaN,OneDropSoup


In [82]:
df_eval.head()

,beer/ABV,beer/name,beer/style,review/appearance,review/aroma,review/palate,review/taste,review/text,user/ageInSeconds,user/birthdayRaw,user/birthdayUnix,user/gender,user/profileName
0,5.1,Roxy Rolles,American Amber / Red Ale,3.0,3.0,3.0,1.5,Disagreeing with the Bros on this one...\t\tAp...,NaN,NaN,NaN,NaN,lilbrewmonkey
1,10.0,Burton Baton,American Double / Imperial IPA,4.5,4.0,5.0,4.0,Love this brew. I'll start with a review of th...,NaN,NaN,NaN,NaN,Tripphop
2,5.6,Old #38 Stout,Irish Dry Stout,4.0,4.0,3.0,4.0,"Dark brown color, medium head. Deeply roasted ...",1.782054e+09,"Jun 23, 1958",-363718800.0,Male,beerguy101
3,7.2,Indian Brown Ale,American Brown Ale,4.5,4.5,4.0,5.0,"Pours a completely black, opaque color, with a...",NaN,NaN,NaN,Male,terrabyte
4,3.6,Greene King IPA Draught Bitter,English Bitter,3.0,2.0,2.5,2.0,Refreshingly lime-infused which is a complimen...,NaN,NaN,NaN,Male,ngandhi


In [83]:
df_dev.describe()

,beer/ABV,review/appearance,review/aroma,review/overall,review/palate,review/taste,user/ageInSeconds,user/birthdayUnix
count,66893.000000,70000.000000,70000.000000,70000.000000,70000.000000,70000.000000,1.464500e+04,1.464500e+04
mean,7.043462,3.845914,3.739307,3.815607,3.747814,3.794571,1.192574e+09,2.257617e+08
std,2.300757,0.611478,0.694185,0.720577,0.679655,0.733947,3.592932e+08,3.592932e+08
min,0.300000,1.000000,1.000000,1.000000,1.000000,1.000000,6.430430e+08,-2.208960e+09
25%,5.250000,3.500000,3.500000,3.500000,3.500000,3.500000,9.837182e+08,1.198620e+08
50%,6.500000,4.000000,4.000000,4.000000,4.000000,4.000000,1.107439e+09,3.108960e+08
75%,8.500000,4.000000,4.000000,4.500000,4.000000,4.500000,1.298473e+09,4.346172e+08
max,41.000000,5.000000,5.000000,5.000000,5.000000,5.000000,3.627295e+09,7.752924e+08


In [84]:
df_eval.describe()

,beer/ABV,review/appearance,review/aroma,review/palate,review/taste,user/ageInSeconds,user/birthdayUnix
count,28715.000000,30000.000000,30000.000000,30000.000000,30000.000000,6.272000e+03,6.272000e+03
mean,7.013776,3.841167,3.737283,3.742917,3.791567,1.191678e+09,2.266572e+08
std,2.296975,0.607496,0.695129,0.684238,0.730495,3.667153e+08,3.667153e+08
min,0.050000,1.000000,1.000000,1.000000,1.000000,5.467934e+08,-2.208960e+09
25%,5.200000,3.500000,3.500000,3.500000,3.500000,9.830270e+08,1.197108e+08
50%,6.500000,4.000000,4.000000,4.000000,4.000000,1.102601e+09,3.157344e+08
75%,8.500000,4.000000,4.000000,4.000000,4.500000,1.298625e+09,4.353084e+08
max,41.000000,5.000000,5.000000,5.000000,5.000000,3.627295e+09,8.715420e+08


In [85]:
df_dev.isna().any(axis=0)

beer/ABV              True
beer/name            False
beer/style           False
review/appearance    False
review/aroma         False
review/overall       False
review/palate        False
review/taste         False
review/text           True
user/ageInSeconds     True
user/birthdayRaw      True
user/birthdayUnix     True
user/gender           True
user/profileName      True
dtype: bool

In [86]:
df_dev.isnull().sum()

beer/ABV              3107
beer/name                0
beer/style               0
review/appearance        0
review/aroma             0
review/overall           0
review/palate            0
review/taste             0
review/text             18
user/ageInSeconds    55355
user/birthdayRaw     55355
user/birthdayUnix    55355
user/gender          41819
user/profileName        14
dtype: int64

In [87]:
df_eval.isnull().sum()

beer/ABV              1285
beer/name                0
beer/style               0
review/appearance        0
review/aroma             0
review/palate            0
review/taste             0
review/text              4
user/ageInSeconds    23728
user/birthdayRaw     23728
user/birthdayUnix    23728
user/gender          17870
user/profileName         3
dtype: int64

In [88]:
df = pd.concat([df_dev, df_eval], sort=False)
len(df_dev), len(df_eval), len(df)

(70000, 30000, 100000)

In [89]:
df.isna().any(axis=0)

beer/ABV              True
beer/name            False
beer/style           False
review/appearance    False
review/aroma         False
review/overall        True
review/palate        False
review/taste         False
review/text           True
user/ageInSeconds     True
user/birthdayRaw      True
user/birthdayUnix     True
user/gender           True
user/profileName      True
dtype: bool

In [90]:
df.isnull().sum()

beer/ABV              4392
beer/name                0
beer/style               0
review/appearance        0
review/aroma             0
review/overall       30000
review/palate            0
review/taste             0
review/text             22
user/ageInSeconds    79083
user/birthdayRaw     79083
user/birthdayUnix    79083
user/gender          59689
user/profileName        17
dtype: int64

In [91]:
(df[df["user/ageInSeconds"].isna()].index == df[df["user/birthdayRaw"].isna()].index).all(),\
(df[df["user/ageInSeconds"].isna()].index == df[df["user/birthdayUnix"].isna()].index).all(),\
(df[df["user/birthdayRaw"].isna()].index == df[df["user/birthdayUnix"].isna()].index).all()


(True, True, True)

In [92]:
from sklearn.model_selection import train_test_split

df_dropped = df.drop(columns=["beer/name", "beer/style", "beer/ABV", "review/text","user/ageInSeconds" ,"user/birthdayRaw", "user/birthdayUnix", "user/gender", "user/profileName"])

train_valid_mask = ~df_dropped["review/overall"].isna()

X = df_dropped.drop(columns=["review/overall"]).values

y = df_dropped["review/overall"].values

X_train_valid = X[train_valid_mask]
y_train_valid = y[train_valid_mask]
X_test = X[~train_valid_mask]
y_test = y[~train_valid_mask]
X_train, X_valid, y_train, y_valid = train_test_split(X_train_valid, y_train_valid, shuffle=True)

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
reg = RandomForestRegressor(100)
reg.fit(X_train , y_train)
r2_score(y_valid, reg.predict(X_valid))

0.6590498221111216

In [93]:
df["user/ageInSeconds"]. fillna(7.043462, inplace=True)

In [94]:
df.head()

,beer/ABV,beer/name,beer/style,review/appearance,review/aroma,review/overall,review/palate,review/taste,review/text,user/ageInSeconds,user/birthdayRaw,user/birthdayUnix,user/gender,user/profileName
0,5.70,Barney Flats Oatmeal Stout,Oatmeal Stout,4.5,5.0,4.5,4.0,4.5,Pours a dark almost cola color with reddish br...,1.066918e+09,"Feb 19, 1981",351417600.0,Male,goblue3509
1,9.99,Brown Shugga',American Strong Ale,4.0,4.0,4.0,4.5,5.0,I went to a place called World Market in Oxnar...,7.043462e+00,NaN,NaN,NaN,Lounge14ABV
2,5.35,Casta Bruna (Pale Ale),English Pale Ale,3.0,3.0,4.0,3.0,3.5,I clicked on Casta tonight just to add some no...,7.043462e+00,NaN,NaN,NaN,DogFood11
3,8.00,Old Engine Oil Special Reserve (Aged In Malt W...,Old Ale,4.0,4.5,4.0,5.0,4.0,Wonderful little beer. Pours a deep brown with...,7.043462e+00,NaN,NaN,NaN,BasementDweller
4,7.00,Dark Humor,American Porter,3.5,4.0,4.0,4.0,4.5,Excited to see this on tap at Fathead's origin...,7.043462e+00,NaN,NaN,NaN,OneDropSoup


In [95]:
df.isnull().sum()

beer/ABV              4392
beer/name                0
beer/style               0
review/appearance        0
review/aroma             0
review/overall       30000
review/palate            0
review/taste             0
review/text             22
user/ageInSeconds        0
user/birthdayRaw     79083
user/birthdayUnix    79083
user/gender          59689
user/profileName        17
dtype: int64

In [96]:
df["beer/ABV"]. fillna(7.043462, inplace=True)

In [97]:
df.isnull().sum()

beer/ABV                 0
beer/name                0
beer/style               0
review/appearance        0
review/aroma             0
review/overall       30000
review/palate            0
review/taste             0
review/text             22
user/ageInSeconds        0
user/birthdayRaw     79083
user/birthdayUnix    79083
user/gender          59689
user/profileName        17
dtype: int64

In [98]:
from sklearn.model_selection import train_test_split

df_dropped = df.drop(columns=["beer/ABV","user/gender", "beer/style","user/ageInSeconds","beer/name", "review/text", "user/profileName","user/birthdayRaw","user/birthdayUnix"])

train_valid_mask = ~df_dropped["review/overall"].isna()

feature_names = df_dropped[train_valid_mask].drop(columns=["review/overall"]).columns

X = df_dropped.drop(columns=["review/overall"]).values

y = df_dropped["review/overall"].values

X_train_valid = X[train_valid_mask]
y_train_valid = y[train_valid_mask]
X_test = X[~train_valid_mask]
y_test = y[~train_valid_mask]
X_train, X_valid, y_train, y_valid = train_test_split(X_train_valid, y_train_valid, shuffle=True, random_state=42)

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
reg = RandomForestRegressor(100, random_state=42)
reg.fit(X_train , y_train)
r2_score(y_valid, reg.predict(X_valid))

0.6573432880926039

In [99]:
df["user/birthdayRaw"]. fillna(0, inplace=True)

In [100]:
from sklearn.model_selection import train_test_split

df_dropped = df.drop(columns=["beer/ABV","user/gender", "beer/style","user/ageInSeconds","beer/name", "review/text", "user/profileName","user/birthdayRaw","user/birthdayUnix"])

train_valid_mask = ~df_dropped["review/overall"].isna()

feature_names = df_dropped[train_valid_mask].drop(columns=["review/overall"]).columns

X = df_dropped.drop(columns=["review/overall"]).values

y = df_dropped["review/overall"].values

X_train_valid = X[train_valid_mask]
y_train_valid = y[train_valid_mask]
X_test = X[~train_valid_mask]
y_test = y[~train_valid_mask]
X_train, X_valid, y_train, y_valid = train_test_split(X_train_valid, y_train_valid, shuffle=True, random_state=42)

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
reg = RandomForestRegressor(100, random_state=42)
reg.fit(X_train , y_train)
r2_score(y_valid, reg.predict(X_valid))

0.6573432880926039

In [101]:
sorted(zip(feature_names, reg.feature_importances_), key=lambda x: x[1], reverse=True)

[('review/taste', 0.8966925724185741),
 ('review/palate', 0.05732173520524936),
 ('review/appearance', 0.023338935346328145),
 ('review/aroma', 0.02264675702984824)]

In [102]:
df.isnull().sum()


beer/ABV                 0
beer/name                0
beer/style               0
review/appearance        0
review/aroma             0
review/overall       30000
review/palate            0
review/taste             0
review/text             22
user/ageInSeconds        0
user/birthdayRaw         0
user/birthdayUnix    79083
user/gender          59689
user/profileName        17
dtype: int64

In [103]:
df["user/birthdayUnix"]. fillna(0, inplace=True)

In [104]:
df_1h = pd.get_dummies(df, columns=["user/gender"])

from sklearn.model_selection import train_test_split

df_dropped = df_1h.drop(columns=["beer/ABV","beer/style","user/ageInSeconds","beer/name", "review/text", "user/profileName","user/birthdayRaw","user/birthdayUnix"])

train_valid_mask = ~df_dropped["review/overall"].isna()

feature_names = df_dropped[train_valid_mask].drop(columns=["review/overall"]).columns

X = df_dropped.drop(columns=["review/overall"]).values

y = df_dropped["review/overall"].values

X_train_valid = X[train_valid_mask]
y_train_valid = y[train_valid_mask]
X_test = X[~train_valid_mask]
y_test = y[~train_valid_mask]
X_train, X_valid, y_train, y_valid = train_test_split(X_train_valid, y_train_valid, shuffle=True, random_state=42)

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
reg = RandomForestRegressor(100, random_state=42)
reg.fit(X_train , y_train)
r2_score(y_valid, reg.predict(X_valid))

0.651063296151476

In [105]:
sorted(zip(feature_names, reg.feature_importances_), key=lambda x: x[1], reverse=True)

[('review/taste', 0.876924277608148),
 ('review/palate', 0.0570262916098255),
 ('review/appearance', 0.027414773205761064),
 ('review/aroma', 0.026666665486892276),
 ('user/gender_Male', 0.009600041499715214),
 ('user/gender_Female', 0.00236795058965803)]

In [106]:
df["user/gender"]. fillna(0, inplace=True)

In [107]:


from sklearn.model_selection import train_test_split

df_dropped = df.drop(columns=["user/gender", "beer/ABV","beer/style","user/ageInSeconds","beer/name", "review/text", "user/profileName","user/birthdayRaw","user/birthdayUnix"])

train_valid_mask = ~df_dropped["review/overall"].isna()

feature_names = df_dropped[train_valid_mask].drop(columns=["review/overall"]).columns

X = df_dropped.drop(columns=["review/overall"]).values

y = df_dropped["review/overall"].values

X_train_valid = X[train_valid_mask]
y_train_valid = y[train_valid_mask]
X_test = X[~train_valid_mask]
y_test = y[~train_valid_mask]
X_train, X_valid, y_train, y_valid = train_test_split(X_train_valid, y_train_valid, shuffle=True, random_state=42)

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
reg = RandomForestRegressor(100, random_state=42)
reg.fit(X_train , y_train)
r2_score(y_valid, reg.predict(X_valid))

0.6573432880926039

In [108]:
df_1h = pd.get_dummies(df, columns=["beer/style"])

from sklearn.model_selection import train_test_split

df_dropped = df_1h.drop(columns=["user/gender", "user/ageInSeconds","beer/name", "review/text", "user/profileName","user/birthdayRaw","user/birthdayUnix"])

train_valid_mask = ~df_dropped["review/overall"].isna()

feature_names = df_dropped[train_valid_mask].drop(columns=["review/overall"]).columns

X = df_dropped.drop(columns=["review/overall"]).values

y = df_dropped["review/overall"].values

X_train_valid = X[train_valid_mask]
y_train_valid = y[train_valid_mask]
X_test = X[~train_valid_mask]
y_test = y[~train_valid_mask]
X_train, X_valid, y_train, y_valid = train_test_split(X_train_valid, y_train_valid, shuffle=True, random_state=42)

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
reg = RandomForestRegressor(100, random_state=42)
reg.fit(X_train , y_train)
r2_score(y_valid, reg.predict(X_valid))

0.6183739102365546

In [109]:
df_1h = pd.get_dummies(df, columns=["beer/style"])

from sklearn.model_selection import train_test_split

df_dropped = df_1h.drop(columns=["beer/ABV", "user/gender", "user/ageInSeconds","beer/name", "review/text", "user/profileName","user/birthdayRaw","user/birthdayUnix"])

train_valid_mask = ~df_dropped["review/overall"].isna()

feature_names = df_dropped[train_valid_mask].drop(columns=["review/overall"]).columns

X = df_dropped.drop(columns=["review/overall"]).values

y = df_dropped["review/overall"].values

X_train_valid = X[train_valid_mask]
y_train_valid = y[train_valid_mask]
X_test = X[~train_valid_mask]
y_test = y[~train_valid_mask]
X_train, X_valid, y_train, y_valid = train_test_split(X_train_valid, y_train_valid, shuffle=True, random_state=42)

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
reg = RandomForestRegressor(100, random_state=42)
reg.fit(X_train , y_train)
r2_score(y_valid, reg.predict(X_valid))

0.6238278786296673

In [110]:
df["user/profileName"]. fillna("sina", inplace=True)

In [111]:
df.isnull().sum()

beer/ABV                 0
beer/name                0
beer/style               0
review/appearance        0
review/aroma             0
review/overall       30000
review/palate            0
review/taste             0
review/text             22
user/ageInSeconds        0
user/birthdayRaw         0
user/birthdayUnix        0
user/gender              0
user/profileName         0
dtype: int64

In [112]:
from sklearn.feature_extraction.text import TfidfVectorizer
#stop_words = ["beer", 'and', "the", "english","to", "is", "this","but","in","that", "for", "on", "some", "that"]
vectorizer = TfidfVectorizer(stop_words = "english", binary=True, use_idf=False, norm=False)
wpm = vectorizer.fit_transform(df_1h["review/text"].fillna(""))

In [113]:
N = 150
freq = sorted(zip(vectorizer.get_feature_names(), wpm.sum(axis=0).tolist()[0]), key=lambda x: x[1], reverse=True)[:N]
freq

[('head', 84804.0),
 ('beer', 59347.0),
 ('taste', 53629.0),
 ('good', 41768.0),
 ('carbonation', 41411.0),
 ('nice', 40343.0),
 ('light', 39120.0),
 ('malt', 39034.0),
 ('like', 38383.0),
 ('sweet', 38025.0),
 ('color', 36028.0),
 ('hops', 35578.0),
 ('aroma', 35383.0),
 ('pours', 34407.0),
 ('mouthfeel', 34276.0),
 ('flavor', 33136.0),
 ('lacing', 32767.0),
 ('white', 32523.0),
 ('finish', 31389.0),
 ('smell', 31181.0),
 ('bit', 31153.0),
 ('glass', 31149.0),
 ('bottle', 29701.0),
 ('medium', 29684.0),
 ('dark', 29129.0),
 ('little', 28542.0),
 ('body', 27705.0),
 ('poured', 26346.0),
 ('brown', 25834.0),
 ('just', 25310.0),
 ('hop', 23707.0),
 ('really', 21837.0),
 ('nose', 21637.0),
 ('smooth', 21625.0),
 ('caramel', 21131.0),
 ('bitterness', 20922.0),
 ('alcohol', 20634.0),
 ('great', 19711.0),
 ('notes', 18866.0),
 ('bitter', 18765.0),
 ('amber', 18623.0),
 ('overall', 18512.0),
 ('malts', 18183.0),
 ('sweetness', 18148.0),
 ('flavors', 17903.0),
 ('citrus', 17612.0),
 ('slightly

In [114]:
import numpy as np

words = [ word for word, _ in freq ]
mask = [ w in words for w in vectorizer.get_feature_names() ]
words_ = [ w for w in vectorizer.get_feature_names() if w in words ]
words_df = pd.DataFrame(data=wpm[:, np.array(mask)].toarray(), columns=[f"word_{word}" for word in words_], index=df_1h.index)

In [115]:
df_1h = df.join(pd.DataFrame(data=wpm[:, np.array(mask)].toarray(), columns=[f"word_{word}" for word in words_], index=df_1h.index))

df_dropped = df_1h.drop(columns=["user/gender", "beer/ABV","beer/style","user/ageInSeconds","beer/name", "review/text", "user/profileName","user/birthdayRaw","user/birthdayUnix"])

train_valid_mask = ~df_dropped["review/overall"].isna()
feature_names = df_dropped[train_valid_mask].drop(columns=["review/overall"]).columns
X = df_dropped.drop(columns=["review/overall"]).values
y = df_dropped["review/overall"].values
X_train_valid = X[train_valid_mask]
y_train_valid = y[train_valid_mask]
X_test = X[~train_valid_mask]
y_test = y[~train_valid_mask]
X_train, X_valid, y_train, y_valid = train_test_split(X_train_valid, y_train_valid, shuffle=True, random_state=42)
reg = RandomForestRegressor(100, random_state=42)
reg.fit(X_train, y_train)
r2_score(y_valid, reg.predict(X_valid))

0.6715670883223617

In [116]:
y_valid.shape

(25000,)

In [117]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words = "english", binary=True, use_idf=False, norm=False)
wpm2 = vectorizer.fit_transform(df_1h["beer/name"].fillna(""))

In [118]:
N = 150
freq2 = sorted(zip(vectorizer.get_feature_names(), wpm2.sum(axis=0).tolist()[0]), key=lambda x: x[1], reverse=True)[:N]
freq2

[('ale', 38573.0),
 ('stout', 14006.0),
 ('ipa', 11472.0),
 ('pale', 9392.0),
 ('imperial', 7682.0),
 ('porter', 6465.0),
 ('lager', 5257.0),
 ('samuel', 4306.0),
 ('beer', 4244.0),
 ('black', 4110.0),
 ('old', 3701.0),
 ('double', 3689.0),
 ('india', 3637.0),
 ('red', 3544.0),
 ('adams', 3438.0),
 ('brown', 3397.0),
 ('hop', 2959.0),
 ('wheat', 2760.0),
 ('style', 2625.0),
 ('stone', 2418.0),
 ('amber', 2270.0),
 ('sierra', 2191.0),
 ('nevada', 2183.0),
 ('aged', 2132.0),
 ('oatmeal', 2094.0),
 ('bell', 2093.0),
 ('barrel', 2080.0),
 ('dark', 2048.0),
 ('winter', 1991.0),
 ('founders', 1984.0),
 ('white', 1860.0),
 ('st', 1790.0),
 ('bock', 1788.0),
 ('belgian', 1700.0),
 ('series', 1698.0),
 ('anniversary', 1652.0),
 ('saison', 1645.0),
 ('special', 1640.0),
 ('extra', 1606.0),
 ('chocolate', 1584.0),
 ('big', 1455.0),
 ('pilsner', 1429.0),
 ('blue', 1405.0),
 ('tripel', 1381.0),
 ('great', 1380.0),
 ('bourbon', 1364.0),
 ('golden', 1330.0),
 ('reserve', 1320.0),
 ('la', 1312.0),
 ('

In [119]:
import numpy as np

words = [ word for word, _ in freq2 ]
mask = [ w in words for w in vectorizer.get_feature_names() ]
words_ = [ w for w in vectorizer.get_feature_names() if w in words ]
words_df = pd.DataFrame(data=wpm2[:, np.array(mask)].toarray(), columns=[f"word_{word}" for word in words_], index=df_1h.index)

In [120]:
df_1h = df.join(pd.DataFrame(data=wpm2[:, np.array(mask)].toarray(), columns=[f"word_{word}" for word in words_], index=df_1h.index))
df_dropped = df_1h.drop(columns=["user/gender", "beer/ABV","beer/style","user/ageInSeconds","beer/name", "review/text", "user/profileName","user/birthdayRaw","user/birthdayUnix"])

train_valid_mask = ~df_dropped["review/overall"].isna()
feature_names = df_dropped[train_valid_mask].drop(columns=["review/overall"]).columns
X = df_dropped.drop(columns=["review/overall"]).values
y = df_dropped["review/overall"].values
X_train_valid = X[train_valid_mask]
y_train_valid = y[train_valid_mask]
X_test = X[~train_valid_mask]
y_test = y[~train_valid_mask]
X_train, X_valid, y_train, y_valid = train_test_split(X_train_valid, y_train_valid, shuffle=True, random_state=42)
reg = RandomForestRegressor(100, random_state=42)
reg.fit(X_train, y_train)
r2_score(y_valid, reg.predict(X_valid))

0.7192349745510798

In [ ]:
#from sklearn.model_selection import GridSearchCV
#param_grid = {
#"n_estimators": [100, 250, 500],
#"criterion": ["mse", "mae"],
#"max_features": ["auto", "sqrt", "log2"],
#"random_state": [42], 
#"n_jobs": [-1], 
#}
#gs = GridSearchCV(RandomForestRegressor(), param_grid, scoring="r2", n_jobs=-1, cv=5)
#gs.fit(X_train_valid, y_train_valid)
#gs.best_score_

In [ ]:
#y_pred = gs.predict(X_test)
#pd.DataFrame(y_pred, index=df[~train_valid_mask].index).to_csv("output.csv", index_label="Id", header=["Predicted"])